# Basic BoW model 

In [1]:
from collections import defaultdict
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from data import get_data
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def cleanup(x):
    # removes punctuation
    x = re.sub(r'[.,"!#/]+', ' ', x, flags=re.MULTILINE)
    # removs \\n
    x = re.sub(r'\\n', ' ', x, flags=re.MULTILINE)
    # removes retweets (e.g. @FER)
    x = re.sub(r'@\w+', '', x, flags=re.MULTILINE)

    return x

In [3]:
def vectorize_X(tweet, vocab):
    # obsolete 
    # tweet = re.sub(r'[.,"!#]+', ' ', tweet, flags=re.MULTILINE)
    #tweet = re.sub(r'@\w+', ' ', tweet, flags=re.MULTILINE)

    # splits tags into 2, e.g. #happy -> #, happy
    # TODO: tags could be useful for classification
    tweet_toks = word_tokenize(tweet)
    X_ = []
    for w in vocab:
        X_.append(1 if w in tweet_toks else 0)

    return X_

In [4]:
def create_vocab(tweets):
    vocab = set()
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    #stemmer = SnowballStemmer()
    
    clean_tweets = [cleanup(t) for t in tweets]
    
    for t in clean_tweets:
        for w in word_tokenize(t):
            vocab.add(w.strip())
    
    vocab -= set(stop_words)
        
    return sorted(vocab)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import accuracy_score

In [6]:
X_train, y_train = get_data('data/2018-E-c-En-train.txt')
X_test, y_test = get_data('data/2018-E-c-En-dev.txt')

X_train_clean = [cleanup(x) for x in X_train]
X_test_clean = [cleanup(x) for x in X_test]

#vocab = create_vocab(X)

#vectorizer = CountVectorizer(ngram_range=(1, 4), stop_words='english')#, vocabulary=vocab)
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=500)
#reducer = TruncatedSVD(n_components=300, random_state=42)
normalizer = Normalizer()
clf = OneVsRestClassifier(LinearSVC(random_state=42))

estimators = [('vectorize', vectorizer), ('normalize', normalizer), ('clf', clf)]

print(X_train_clean[0])
vectorizer.fit(X_train_clean)
print(vectorizer.transform([X_train_clean[0]]))

“Worry is a down payment on a problem you may never have'   Joyce Meyer    motivation  leadership  worry
  (0, 487)	0.8676948562339434
  (0, 334)	0.4970972102769805


In [7]:
X_train, y_train = get_data('data/2018-E-c-En-train.txt')
X_test, y_test = get_data('data/2018-E-c-En-dev.txt')

X_train_clean = np.array([cleanup(x) for x in X_train])
X_test_clean = np.array([cleanup(x) for x in X_test])

#vocab = create_vocab(X)

#vectorizer = CountVectorizer(ngram_range=(1, 4), stop_words='english')#, vocabulary=vocab)
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=500)
#reducer = TruncatedSVD(n_components=300, random_state=42)
normalizer = Normalizer()
clf = OneVsRestClassifier(LinearSVC(random_state=42))

estimators = [('vectorize', vectorizer), ('normalize', normalizer), ('clf', clf)]

print(X_train_clean[0])
vectorizer.fit(X_train_clean)
print(vectorizer.transform(X_train_clean))
#print(X_train_clean.shape)

“Worry is a down payment on a problem you may never have'   Joyce Meyer    motivation  leadership  worry
  (0, 487)	0.8676948562339434
  (0, 334)	0.4970972102769805
  (1, 408)	0.5560172132022704
  (1, 269)	0.5264108866647862
  (1, 268)	0.4660600855948308
  (1, 179)	0.44331076418119336
  (2, 491)	0.7300553112533512
  (2, 322)	0.6833880614341843
  (3, 244)	0.5876297457788291
  (3, 122)	0.4800650057356455
  (3, 6)	0.6513285439345825
  (4, 417)	0.6434071602402384
  (4, 302)	0.765524151252978
  (5, 376)	1.0
  (6, 426)	1.0
  (7, 435)	0.3673646200452048
  (7, 217)	0.41166901455258914
  (7, 139)	0.7210813064173454
  (7, 103)	0.41906277325932556
  (8, 449)	0.5012580617338288
  (8, 333)	0.48405292781672216
  (8, 197)	0.5418983540235066
  (8, 86)	0.46987157024608195
  (9, 162)	1.0
  (10, 377)	0.5989722027390078
  :	:
  (6831, 397)	0.8255622334051632
  (6831, 179)	0.5643110833353167
  (6832, 497)	0.6922247929503843
  (6832, 334)	0.7216819493549756
  (6833, 485)	0.6949768682634876
  (6833, 74)	0.71

In [8]:
print()

In [9]:
pipe = Pipeline(memory=None, steps=estimators).fit(X_train_clean, y_train)

print(accuracy_score(y_test, pipe.predict(X_test_clean)))
y_rand = np.array([np.round(np.random.uniform(0, 1, y_test.shape[1])) for _ in range(y_test.shape[0])])
print(accuracy_score(y_test, y_rand))




0.19187358916478556
0.002257336343115124


In [11]:
from sklearn.preprocessing import OneHotEncoder
from data import get_data

from keras.utils.np_utils import to_categorical

X_train, y_train = get_data('data/2018-EI-oc-En-anger-dev.txt')

print(to_categorical(y_train))
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1))
print(y_train)

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.python.training'